# Sample Mass-Difference Networks in Metabolomics Data Analysis

Notebook to support the study on the application of **Sample M**ass-**Di**fference **N**etworks as a highly specific competing form of pre-processing procedure for high-resolution metabolomics data.

Mass-Difference Networks are focused into making networks from a list of masses. Each _m/z_ will represent a node. Nodes will be connected if the difference in their masses can be associated to a simple chemical reaction (enzymatic or non-enzymatic) that led to a change in the elemental composition of its metabolite.

The set of mass differences used to build said networks are called a set of MDBs - Mass-Difference-based Building block.

This is notebook `paper_sMDiNs_database_prep.ipynb`.


## Organization of the Notebook

- **Set up database of the 7 benchmark datasets**
- Description of the benchmark datasets
- **Building transformation list (of MDBs) and setting up list of masses files to use in Cytoscape's MetaNetter (to build MDiNs)**
- Show the benchmark datasets characteristics
- Explanation of the parameters used in Cytoscape's MetaNetter 2.0 to build the MDiNs.
- Apply intensity-based pre-treatments.
- **Store pre-processed and pre-treated data**


#### Needed Imports

In [1]:
import numpy as np
import pandas as pd
from pandas.testing import assert_frame_equal

# Metabolinks package
import metabolinks as mtl
import metabolinks.transformations as transf

from pathlib import Path
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [2]:
%matplotlib inline

In [3]:
# json for persistence
import json
from time import perf_counter

In [4]:
# Atomic masses - https://ciaaw.org/atomic-masses.htm
#Isotopic abundances-https://ciaaw.org/isotopic-abundances.htm/https://www.degruyter.com/view/journals/pac/88/3/article-p293.xml
# Isotopic abundances from Pure Appl. Chem. 2016; 88(3): 293–306,
# Isotopic compositions of the elements 2013 (IUPAC Technical Report), doi: 10.1515/pac-2015-0503

chemdict = {'H':(1.0078250322, 0.999844),
            'C':(12.000000000, 0.988922),
            'N':(14.003074004, 0.996337),
            'O':(15.994914619, 0.9976206),
            'Na':(22.98976928, 1.0),
            'P':(30.973761998, 1.0),
            'S':(31.972071174, 0.9504074),
            'Cl':(34.9688527, 0.757647),
            'F':(18.998403163, 1.0),
            'C13':(13.003354835, 0.011078) # Carbon 13 isotope
           } 

# electron mass from NIST http://physics.nist.gov/cgi-bin/cuu/Value?meu|search_for=electron+mass
electron_mass = 0.000548579909065

## Description of dataset records

`datasets` is the global dict that holds all data sets. It is a **dict of dict's**.

Each data set is **represented as a dict**.

Each record has the following fields (keys):

- `name`: the table/figure name of the data set
- `source`: the biological source for each dataset
- `mode`: the aquisition mode
- `alignment`: the alignment used to generate the data matrix
- `data`: the data matrix
- `target`: the sample labels, possibly already integer encoded
- `MDiN`: Mass-Difference Network - Not present here, only on sMDiNsAnalysis notebook
- `<treatment name>`: transformed data matrix / network. These treatment names can be
    - `original`: an alias to `data`
    - `Ionly`: missing value imputed data by 1/5 of the minimum value in each sample in the dataset, only
    - `P`: Pareto scaled data
    - `NP`: Pareto scaled and normalized
    - `NGP`: normalized, glog transformed and Pareto scaled
    - `Ionly_RF`: missing value imputed data by random forests, only
    - `P_RF`: Pareto scaled data
    - `NP_RF`: Pareto scaled and normalized
    - `NGP_RF`: normalized, glog transformed and Pareto scaled
    - `IDT`: `NGP_RF` or `NGP` - Intensity-based Data pre-Treatment chosen as comparison based on which of the two performed better for each dataset and each statistical method
    - `sMDiN`: Sample Mass-Difference Networks - Not present here, only on sMDiNsAnalysis notebook
       
- `<sMDiN analysis name>`: data matrix from nework analysis of MDiNs - Not in this notebook
    - `Degree`: degree analysis of each sMDiN
    - `Betweenness`: betweenness centrality analysis of each sMDiN
    - `Closeness`: closeness centrality of analysis of each sMDiN
    - `MDBI`: analysis on the impact of each MDB (Mass-Difference based building-block) on building each sMDiN
    - `GCD11`: Graphlet Correlation Distance of 11 different orbits (maximum of 4-node graphlets) between each sMDiN.

The keys of `datasets` may be shared with dicts holding records resulting from comparison analysis.

Here are the keys (and respective names) of datasets used in this study:

- GD_neg_global2 (GDg2-)
- GD_pos_global2 (GDg2+)
- GD_neg_class2 (GDc2-)
- GD_pos_class2 (GDc2+)
- YD (YD)
- vitis_types (GD types)
- HD (HD)

### Description of grapevine data sets

Grapevine Datasets (Negative and Positive) - 33 samples belonging to 11 different grapevine varieties (3 samples per variety/biological group) of FT-ICR-MS metabolomics data obtained in negative and positive ionization mode.

5 different _Vitis_ species (other than _V. vinifera_) varieties:

- CAN - 3 Samples (14, 15, 16) of _V. candicans Engelmann_ (VIVC variety number: 13508)
- RIP - 3 Samples (17, 18, 19) of _V. riparia Michaux_ (Riparia Gloire de Montpellier, VIVC variety number: 4824) 
- ROT - 3 Samples (20, 21, 22) of _V. rotundifolia_ (Muscadinia Rotundifolia Michaux cv. Rotundifolia, VIVC variety number: 13586)
- RU - 3 Samples (35, 36, 37) of _V. rupestris Scheele_ (Rupestris du lot, VIVC variety number: 10389)
- LAB - 3 Samples (8, 9, 10) of _V. labrusca_ (Isabella, VIVC variety number: 5560)

6 different _V. vinifera_ cultivars varieties are:

- SYL - 3 samples (11, 12, 13) of the subspecies _sylvestris_ (VIVC variety number: -)
- CS - 3 Samples (29, 30, 31) of the subspecies _sativa_ cultivar Cabernet Sauvignon (VIVC variety number: 1929)
- PN - 3 Samples (23, 24, 25) of the subspecies _sativa_ cultivar Pinot Noir (VIVC variety number: 9279)
- REG - 3 Samples (38, 39, 40) of the subspecies _sativa_ cultivar Regent (VIVC variety number: 4572)
- RL - 3 Samples (26, 27, 28) of the subspecies _sativa_ cultivar Riesling Weiss (VIVC variety number: 10077)
- TRI - 3 Samples (32, 33, 34) of the subspecies _sativa_ cultivar Cabernet Sauvignon (VIVC variety number: 15685)

Data acquired by Maia et al. (2020):

- Maia M, Ferreira AEN, Nascimento R, et al. Integrating metabolomics and targeted gene expression to uncover potential biomarkers of fungal / oomycetes ‑ associated disease susceptibility in grapevine. Sci Rep. Published online 2020:1-15. doi:10.1038/s41598-020-72781-2
- Maia M, Figueiredo A, Silva MS, Ferreira A. Grapevine untargeted metabolomics to uncover potential biomarkers of fungal/oomycetes-associated diseases. 2020. doi:10.6084/m9.figshare.12357314.v1

**Peak Alignment** and **Peak Filtering** were performed with function `metabolinks.peak_alignment.align()`. Human leucine enkephalin (Sigma Aldrich) was used as the reference feature (internal standard, [M+H]+ = 556.276575 Da or [M-H]- = 554.262022 Da).

**4** data matrices were constructed from this data:

- Data sets named `GD_pos_global2` (GDg2+) and `GD_neg_global2` (GDg2-) were generated after retaining only features that occur (globally) at least twice in all 33 samples of the data sets (filtering/alignment) for the **positive mode** data acquisition and the **negative mode** data acquisition, respectively.
- Data sets named `GD_pos_class2` (GDc2+) and `GD_neg_class2` (GDc2-) were generated after retaining only features that occur at least twice in the three replicates of at least one _Vitis_ variety in the data sets (filtering/alignment) for the **positive mode** data acquisition and the **negative mode** data acquisition, respectively.

For the purpose of assessing the performance of supervised methods each of these four datasets was used with target labels defining classes corresponding to replicates of each of the 11 Vitis species/cultivars.

For the purpose of assessing the performance of supervised methods under a binary (two-class) problem, data set `GD_neg_class2` was also used with target labels defining two classes: Vitis vinifera cultivars and "wild", non-vinifera Vitis species. This is dataset `vitis_types` (GD types).

### Description of the yeast data set

Yeast dataset - 15 samples belonging to 5 different yeast strains of _Saccharomyces cerevisiae_ (3 biological replicates per strain/biological group) of FT-ICR-MS metabolomics data obtained in positive ionization mode. The 5 strains were: the reference strain BY4741 (represented as BY) and 4 single-gene deletion mutants of this strain – ΔGLO1 (GLO1), ΔGLO2 (GLO2), ΔGRE3 (GRE3) and ΔENO1 (ENO1). These deleted genes are directly or indirectly related to methylglyoxal metabolism.

Data acquired by Luz et al. (2021):

- Luz J, Pendão AS, Silva MS, Cordeiro C. FT-ICR-MS based untargeted metabolomics for the discrimination of yeast mutants. 2021. doi:10.6084/m9.figshare.15173559.v1

**Peak Alignment** and **Peak Filtering** was performed with MetaboScape 4.0 software (see reference above for details in sample preparation, pre-processing, formula assignment). In short, Yeast Dataset was obtained with Electrospray Ionization in Positive Mode and pre-processed by MetaboScape 4.0 (Bruker Daltonics). Human leucine enkephalin (Sigma Aldrich) was used as the reference feature (internal standard, [M+H]+ = 556.276575 Da or [M-H]- = 554.262022 Da).

A data matrix was constructed from this data: data set named **`YD` (YD)** was generated after retaining only features that occur (globally) at least twice in all 15 samples (filtering/alignment).

Furthermore, peaks higher than 1000 m/z were filtered out. Formula assignment was performed with MetaboScape 4.0 software and, to consider peaks that had the same formula assigned, these were joined together in a single peak (`feature_filter` function).

For the purpose of assessing the performance of supervised methods, this data set was used with target labels defining classes corresponding to replicates of each of the 4 yeast strains.

#### Loading yeast data set

In [5]:
def read_MetScape_file(filename,
                          col_renamer=None,
                          add_labels=None,
                          remove_ref_feat=None,):
    
    """Read in a MetaboScape bucket table from a CSV file."""
    
    data = pd.read_csv(filename).set_index('Bucket label')
    
    # optionally rename sample_names
    if col_renamer is not None:
        data = data.rename(columns=renamer)
    
    # optionally remove a reference feature (if already normalized)
    if remove_ref_feat is not None:
        #print(f'Feature {remove_ref_feat}\n{data.loc[remove_ref_feat, :]}\n----------')
        data = data.drop(index=[remove_ref_feat])
        
    # Joining rows that have the same formula assigned (with feature_filter) while keeping rows without formulas assigned.
    data = feature_filter(data)
    
    # split in peak metadata and intensities
    peak_cols = ['m/z', 'Name', 'Formula']
    intensity_cols = [c for c in list(data.columns) if c not in peak_cols]
    peaks = data[peak_cols]
    intensities = data[intensity_cols]

    # replace zeros for NaN's
    intensities = intensities.replace(0, np.nan).dropna(how='all')
    
    # force peaks to have the same features as the (trimmed) intensities
    peaks = peaks.reindex(intensities.index)

    # optionally, add labels to intensities
    if add_labels is not None:
        intensities = mtl.add_labels(intensities, labels=add_labels)
    
    return {'filename': filename,
            'peaks':peaks,
            'intensities': intensities}

def renamer(colname):
    # Util to optionally remove all those 00000 from sample names
    return ''.join(colname.split('00000'))

In [6]:
def feature_filter(Spectra):
    """Join all features that share the same formula by addition.
       
       Spectra: Pandas DataFrame (dataset).
       
       Returns: Pandas DataFrame with filtered dataset.
    """

    df = Spectra
    # Variables to store results
    rp = pd.DataFrame(columns = df.columns)
    rep = []
    # Series with number of times each formula appears in the dataset
    form_count = df['Formula'].value_counts()
    a = 0
    
    for i in df['Formula']:
        if pd.isnull(i) == True: # If no formula is assigned to the feature, the feature stays in the new dataset
            rp = rp.append(df.iloc[a,:])

        else: # If a formula is assigned
            if form_count[i] > 1: # If the formula appears more than one time in the dataset
                if i not in rep: # And If it didn't already appear before
                    
                    # Bucket label, name and formula info are the ones from the greater intensity m/z peak (of peaks with the
                    # same formula)
                    peaks = df.loc[df['Formula'] == i,:].iloc[:,3:].sum(axis = 1)
                    peaks_idx = peaks.idxmax()
                    info = df.loc[peaks_idx].iloc[:3] # Bucket label, name and formula info
                    
                    # Addition of all m/z peaks intensity with the same formula
                    newrow = df.loc[df['Formula'] == i,:].iloc[:,3:].sum(axis = 0)
                    newrow_c = pd.concat([info, newrow], axis = 0) # Join with the m/z peak metadata
                    # m/z peak is from the greater intensity m/z peak (of peaks with the same formula)
                    newrow_c = newrow_c.rename(df.loc[peaks_idx].name)
                    
                    # Append peak with intensity added from all peaks sharing the same formula
                    rp = rp.append(newrow_c)

                    rep.append(i) # Append repeating formulas so this process isn't repeated again for the other m/z peaks

            else:
                rp = rp.append(df.iloc[a,:]) # Append peaks with formulas that aren't repeated in the dataset
        a = a + 1

    # return DataFrame
    return rp

In [7]:
# Labels of the 5 biological groups (5 yeast strains) - only added after
yeast_classes = 'WT ΔGRE3 ΔENO1 ΔGLO1 ΔGLO2'.split()

In [8]:
# Read in the file and keep results in dicts 

prefix_to_drop = None # change to 'ENO' to remove ENO strain

# MetaboScape data
MS_data = read_MetScape_file('5yeasts_notnorm.csv', 
                             remove_ref_feat=None,
                             add_labels=None,
                             col_renamer=renamer)

In [9]:
# keep features that appear in at least two samples
yeast_datamatrix = transf.keep_atleast(MS_data['intensities'].transpose(), minimum=2)

# Changing the index to be numbers - our list of neutral masses from the bucket lists.
new_columns = []
for i in range(len(yeast_datamatrix.columns)):
    new_columns.append(float(yeast_datamatrix.columns[i][:-3]))
yeast_datamatrix.columns = new_columns

# Only keep peaks below 1000 m/z - Very few peaks above 1000 m/z.
yeast_datamatrix = yeast_datamatrix.T[yeast_datamatrix.columns < 1000].T

yeast_datamatrix

,307.083818,555.269298,624.087308,493.316816,257.102875,780.109207,347.063081,254.224610,625.086929,663.109142,...,734.677027,648.574264,634.548871,675.482247,739.516465,916.507669,550.163642,914.598352,802.744007,535.097415
BY0_1,391042880.0,111270160.0,130088936.0,1.122515e+08,54234476.0,33863468.0,35209136.0,9935943.0,25313166.0,21631346.0,...,726365.5625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BY0_2,398003680.0,110423984.0,127133608.0,1.139838e+08,53720908.0,32622178.0,35636648.0,17679414.0,24989162.0,24420090.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BY0_3,399481952.0,111225424.0,131208568.0,1.158259e+08,55826148.0,34655496.0,37943456.0,19530026.0,24684334.0,24862190.0,...,666204.7500,NaN,NaN,NaN,NaN,NaN,558134.1875,NaN,NaN,NaN
GRE3_1,270906400.0,92849040.0,130682664.0,1.104378e+08,46503348.0,33206608.0,32640452.0,5713388.5,25890872.0,28692028.0,...,NaN,NaN,NaN,NaN,NaN,NaN,613780.4375,NaN,NaN,NaN
GRE3_2,271023520.0,94888320.0,130844488.0,1.144803e+08,44828256.0,34597444.0,31696134.0,10139615.0,25548978.0,26964150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,579154.75,NaN
GRE3_3,274854272.0,94514272.0,130896568.0,1.233897e+08,46361588.0,33131336.0,31668180.0,16434200.0,25518902.0,26654308.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENO1_1,255276736.0,150620544.0,136612464.0,3.220034e+07,72465536.0,35298204.0,31236114.0,10759292.0,25798230.0,22221992.0,...,NaN,NaN,720792.625,NaN,NaN,NaN,NaN,NaN,NaN,538023.5625
ENO1_2,251610656.0,148516304.0,134972800.0,3.422184e+07,70376640.0,35198252.0,30789652.0,25896608.0,26849502.0,23433602.0,...,NaN,NaN,NaN,NaN,NaN,626384.625,NaN,609298.9375,NaN,NaN
ENO1_3,251024624.0,150645840.0,135922432.0,3.370513e+07,68912816.0,34380756.0,30916958.0,29723808.0,26606410.0,23500090.0,...,NaN,655682.1875,624387.000,NaN,561889.8750,NaN,NaN,NaN,NaN,NaN
dGLO1_1,283822048.0,202869216.0,130259064.0,4.167029e+07,58269456.0,32095004.0,29669478.0,26538900.0,26013280.0,21245436.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,548121.4375,NaN,NaN


### Description of the Human dataset

Human Dataset - 249 samples belonging to 2 different classes of HILIC-MS data obtained in positive ionization mode. The Chromatography system was a Thermo Dionex Ultimate 3000 with the column being a Waters Xbridge BEH HILIC (75 x 2.1mm, 2.5um). Mass Spectrometry was performed in a Thermo Q Exactive HF hybrid Orbitrap operating in positive electrospray ionization mode. Further information in the data deposition site mentioned below.

The 2 classes are:

- 135 pre-operative blood (serum) samples from patients with **'No Recurrence'** of Prostate Cancer after Radical Prostatectomy
- 114 pre-operative blood (serum) samples from patients with **'Recurrence'** of Prostate Cancer after Radical Prostatectomy

I think this means samples were collected before the Prostatectomy before knowledge of recurrence or no recurrence? There was a total of 80 subjects.

Data taken from data deposition site: https://www.metabolomicsworkbench.org/data/DRCCMetadata.php?Mode=Study&DataMode=AllData&StudyID=ST001082&StudyType=MS&ResultType=5#DataTabs

This data is available at the NIH Common Fund's National Metabolomics Data Repository (NMDR) website, the Metabolomics Workbench, https://www.metabolomicsworkbench.org where it has been assigned Project ID PR000724. The data can be accessed directly via it's Project DOI: 10.21228/M83D5V. 

Data deposited by the Georgia Institute of Technology and researcher Facundo Fernandez. This data was taken already aligned in a 2D numerical data matrix. 

Data used in the paper: Clendinen CS, Gaul DA, Monge ME, et al. Preoperative Metabolic Signatures of Prostate Cancer Recurrence Following Radical  Prostatectomy. J Proteome Res. 2019;18(3):1316-1327. doi:10.1021/acs.jproteome.8b00926

Dataset named **HD (HD)** was generated after retaining only features that occur (globally) at least twice in all samples of the dataset. For the purpose of assessing the performance of supervised methods this dataset was used with target labels defining classes based on which of the 2 classes mentioned prior, the samples belonged to.


The data read from the site seemed to be completely untreated, which was perfect for our efforts. We can't be sure but the presence of plenty of missing values (very useful for us) points to no great peak filtering and missing value imputation being made, the bar plot built from adding up all intensities of each sample seems to point that no normalization had yet been employed in the data, and the distribution observed in the boxplot of the intensities of some of the features in the samples seems to point that no scaling had been used yet in the data as well.

The data also had 2 copies of each sample, where the 2nd copies were equal to the first ones multiplied by a constant. This constant was different for each sample. We removed the presence of the 2nd copies. The index (metabolites) were reduced from the notation m/z value_retention time to only have the m/z value.


#### Loading Human dataset

In [10]:
# Reading the dataset, this dataset has two copies of the samples.The 2nd copy is equal to the 1st multiplied by a constant
# unique to each sample.
human_datamatrix_base = pd.read_excel('ST001082_AN001766_HD.xlsx')

In [11]:
human_datamatrix = human_datamatrix_base.iloc[:-1, :-4] # Just select the rows corresponding to the dataset
mz_list = human_datamatrix_base.iloc[:-1, -4] # Select column with list of m/z values

human_datamatrix = human_datamatrix.set_index(human_datamatrix.columns[0]) # Set index as the metabolites name
human_datamatrix = human_datamatrix.replace({0:np.nan}) # Replacing 0 values as missing values

human_datamatrix

,2_Pooled_Sample_1,2_Pooled_Sample_1_2,P2_2,P2_3,P2_4,Psp_1,Psp_2,Psp_3,Psp_4,Psp_5,...,S67_3.1,S70_1.1,S70_2.1,S70_3.1,S72_1.1,S72_2.1,S72_3.1,S77_1.1,S77_2.1,S77_3.1
metabolite_name,,,,,,,,,,,,,,,,,,,,,
Factors,Sample Type:Pooled,Sample Type:Pooled,Sample Type:Pooled,Sample Type:Pooled,Sample Type:Pooled,Sample Type:Pooled,Sample Type:Pooled,Sample Type:Pooled,Sample Type:Pooled,Sample Type:Pooled,...,Sample Type:No Recurrence,Sample Type:No Recurrence,Sample Type:No Recurrence,Sample Type:No Recurrence,Sample Type:No Recurrence,Sample Type:No Recurrence,Sample Type:No Recurrence,Sample Type:No Recurrence,Sample Type:No Recurrence,Sample Type:No Recurrence
95.06024579_7.857483333,2544.256904,2174.683995,2349.405324,2142.906787,2633.130278,2510.239989,3619.82127,2217.770816,2894.966994,20587.0328,...,4121.029059,3834.23691,4090.323428,3830.272712,3705.545567,3954.147516,3740.035077,3837.308267,4164.971893,4155.190276
71.06031304_0.064166667,153.7382834,2972.450721,951.0420238,2861.700714,3339.380877,299.8124672,783.1155979,3034.247021,3511.131943,NaN,...,8012.257751,853.7856117,6297.664545,7257.02521,845.5966076,6130.01575,7499.72554,796.3862056,5999.547509,7583.908921
71.06031992_9.984466667,292.2381888,1598.405492,638.9111168,1659.297423,1855.933656,382.0675008,948.3162562,1707.100565,1981.102653,62862.60997,...,4440.820011,662.2607137,3347.680062,4147.143726,694.8464783,3388.086581,4194.416655,650.7198089,3381.510189,4134.43322
126.1023924_4.038416667,347.8838312,125.0657553,260.8042864,119.6837125,NaN,803.1992227,NaN,126.2802051,0.001242692,582.9410185,...,322.3904954,4488.12769,3755.93028,1616.363518,1806.669855,1365.45848,495.4602756,1993.623747,1663.073748,1059.509693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588.4147684_2.0983,12264.30083,1935.998098,2697.901871,1761.340872,369.0957455,12243.27287,574.4747938,221.0599403,11.37397281,NaN,...,NaN,6225.892205,NaN,NaN,8491.983882,NaN,NaN,9425.946784,358.953733,NaN
177.1270579_2.0983,4477.875444,317.9191822,2573.952315,69.97337036,NaN,5703.699795,NaN,477.9239747,960.7144488,NaN,...,5104.451502,NaN,248.6690155,4012.400432,189.415007,1938.392231,5601.351328,NaN,25.42248559,4329.675354
374.2064688_2.0983,7056.085863,112.7816239,4444.488636,NaN,NaN,6690.765635,NaN,462.1335736,572.3140362,NaN,...,6158.786604,NaN,NaN,3297.509597,NaN,NaN,2256.379266,NaN,NaN,7587.036926


In [12]:
# Repeating m/z values with different retention times

#abc = []
#for i in list(mz_list):
#    if i in abc:
#        print(i)
#    else:
#        abc.append(i)
#len(set(list(mz_list)))

In [13]:
# Select one of the two copies of samples in the dataset by removing the samples ending with '.1'
human_datamatrix = human_datamatrix[[i for i in human_datamatrix.columns if not i.endswith('.1')]]

human_datamatrix = human_datamatrix.T # Transpose the dataset

In [14]:
# Current sample labels. Sample will consist of 'Sample Type:No Recurrence' and 'Sample Type:Recurrence'
hd_labels = list(human_datamatrix['Factors'])
set(hd_labels)

{'Sample Type:Blank',
 'Sample Type:No Recurrence',
 'Sample Type:Pooled',
 'Sample Type:Recurrence'}

In [15]:
# How many 'No Recurrence' samples in the dataset
hd_labels.count('Sample Type:No Recurrence')

135

### Blank Treatment

- If blank_treatment = True:

An average of the blanks will be subtracted to the remaining dataset. Missing values in the blanks were replaced by 0 to calculate the average of the blanks. Negative values that arise in the dataset from subtracting the blanks will be coded as 0/missing values in the dataset. With the BinSim pre-treatment, they will be coded as 0.

Possible problem for BinSim: features that are very similar between samples might be slightly negative in one and be coded as 0 and slightly positive in the other and be coded as 1, despite being basically identical.

- If blank_treatment = False:

Blank samples are removed and not accounted for. 

In [16]:
blank_treatment = True
#blank_treatment = False

In [17]:
if blank_treatment:
    blanks = human_datamatrix[human_datamatrix['Factors'] == 'Sample Type:Blank'].iloc[:,1:] # Select blank samples
    blanks = blanks.replace({np.nan:0}) 
    blanks = blanks.astype(float) # Get blank samples with floats (needed since datamatrix has strings)
    blanks_average = blanks.mean() # Average of the blanks
    blanks_average

In [18]:
# Selecting the samples belonging to either the 'No Recurrence' or 'Recurrence' 
selection = []
for i in human_datamatrix.loc[:, 'Factors']:
    if i in ['Sample Type:No Recurrence', 'Sample Type:Recurrence']:
        selection.append(True)
    else:
        selection.append(False)

In [19]:
human_datamatrix = human_datamatrix[selection]
# Creating the list of 'targets' (labels of samples) of the dataset with the 'No Recurrence' and 'Recurrence' classes 
hd_labels = []
for i in list(human_datamatrix.iloc[:,0]):
    if i == 'Sample Type:No Recurrence':
        hd_labels.append('No Recurrence')
    else:
        hd_labels.append('Recurrence')

In [20]:
human_datamatrix = human_datamatrix.iloc[:,1:]
human_datamatrix

metabolite_name,95.06024579_7.857483333,71.06031304_0.064166667,71.06031992_9.984466667,126.1023924_4.038416667,121.0717995_1.358816667,105.0697004_2.574533333,83.06023778_3.971433333,85.02828928_2.7925,85.0282975_2.440233333,88.05051567_0.095966667,...,370.0920791_2.0983,468.1567372_2.0983,727.5149425_2.0983,497.1417674_2.0983,400.1388094_2.0983,588.4147684_2.0983,177.1270579_2.0983,374.2064688_2.0983,211.0531259_2.0983,206.0808246_2.0983
S10_1,5839.689646,2421.207099,1702.403305,1732.645968,11202.3156,13.77354446,15969.09873,11122.92882,76033.09098,15559.66213,...,549.5061406,1788.00406,237.4614192,3065.766426,170.0414581,1347.054595,479.0767276,NaN,NaN,41.58211389
S10_2,4925.936713,6760.239327,3927.481184,876.2460627,16464.75568,144.7380541,11235.281,17168.89308,50955.48348,17634.72493,...,2354.284435,428.9492324,6.856593556,126.8752621,6686.085356,NaN,5663.904982,11572.89831,1720.669345,3969.969384
S10_3,5530.737291,11234.04531,6138.510579,302.5141329,24441.42344,196.744245,6281.973729,23329.82436,38668.17711,17924.91677,...,1187.542997,8.16828659,4.535420845,NaN,6985.148407,5.695198924,5001.930045,9622.04692,1835.400914,3534.540622
S15_1,4345.962212,1647.020375,1087.357218,2044.425809,4820.244022,8.539063852,10824.9497,2858.654761,48787.91979,12293.8573,...,NaN,2378.896459,578.7754145,NaN,155.6123097,2164.462213,936.4502838,NaN,NaN,66.89873449
S15_2,4135.848719,7164.394072,3594.221384,1396.827349,6507.869956,56.61978795,8963.260007,6670.576787,49360.26082,23406.46182,...,NaN,437.5625196,11.89033556,1.767176652,1485.254178,73.86922602,1952.194542,NaN,NaN,282.3818514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S72_2,4042.390453,6266.816562,3463.69699,1395.930804,10517.7526,97.700184,17988.63512,4597.778125,24216.15305,23606.86636,...,7049.366359,NaN,0.910517997,1273.048903,3601.773523,NaN,1981.650461,NaN,3938.848802,144.146236
S72_3,3860.581052,7741.451008,4329.607912,511.4295756,15224.61293,594.5983793,9768.49416,7663.919663,17897.95381,12847.39403,...,5875.548618,NaN,0.355728465,NaN,15046.7641,NaN,5781.889838,2329.105172,4675.475887,1257.137318
S77_1,4741.732906,984.0884324,804.0895638,2463.50584,5199.447492,8.686503713,10287.69548,2550.038217,36939.60819,17136.32435,...,NaN,2098.000087,3969.01007,NaN,81.4497374,11647.5714,NaN,NaN,0.896501216,239.696676
S77_2,4153.731532,5983.356024,3372.384222,1658.585471,7551.248853,26.64915706,8092.077171,4522.858963,32505.31818,23193.57557,...,NaN,349.8420486,782.9663571,NaN,486.1349482,357.9849943,25.35387578,NaN,NaN,351.8411172


In [21]:
human_datamatrix = human_datamatrix.astype(float) # Passing the values from strings to floats.

In [22]:
if blank_treatment:
    human_datamatrix = human_datamatrix.replace({np.nan:0}) - blanks_average
    human_datamatrix[human_datamatrix<0] = 0
    human_datamatrix = human_datamatrix.replace({0:np.nan})

In [23]:
#human_datamatrix.columns = list(mz_list)[1:]
#human_datamatrix

Set list of masses to pass to Cytoscape and to build the MDiNs.

'Neutralization' of the _m/z_ values by subtracting the mass of a proton to the _m/z_ peaks.

4 peaks (2 pairs) with different retention time have the exact same _m/z_ values. These 2 pairs of peaks wil be treated as only 2 different peaks (instead of four) for MDiN building purposes.

In [24]:
mz_list = mz_list[1:] - chemdict['H'][0] + electron_mass
counts = human_datamatrix.count(axis=0)
final_mz_list = list(mz_list[list(counts >= 2)])
final_mz_list = set(final_mz_list)

In [25]:
human_datamatrix = transf.keep_atleast(human_datamatrix, minimum=2) # Keep features that appear in at least two samples
human_datamatrix

metabolite_name,71.06031304_0.064166667,71.06031992_9.984466667,126.1023924_4.038416667,121.0717995_1.358816667,105.0697004_2.574533333,83.06023778_3.971433333,85.02828928_2.7925,85.0282975_2.440233333,103.0539927_2.632416667,117.0695881_0.77655,...,370.0920791_2.0983,468.1567372_2.0983,727.5149425_2.0983,497.1417674_2.0983,400.1388094_2.0983,588.4147684_2.0983,177.1270579_2.0983,374.2064688_2.0983,211.0531259_2.0983,206.0808246_2.0983
S10_1,NaN,NaN,767.728678,7664.437238,NaN,11885.760381,11034.772529,71411.834905,NaN,272.371357,...,549.506141,1788.004060,237.461419,3065.766426,170.041458,1347.054595,474.490232,NaN,NaN,41.582114
S10_2,NaN,NaN,NaN,12926.877318,128.638686,7151.942651,17080.736789,46334.227405,881.713011,470.898334,...,2354.284435,428.949232,6.856594,126.875262,6686.085356,NaN,5659.318486,11569.699041,1720.669345,3969.969384
S10_3,1546.96215,0.29663,NaN,20903.545078,180.644877,2198.635380,23241.668069,34046.921035,1439.412175,101.298661,...,1187.542997,8.168287,4.535421,NaN,6985.148407,5.695199,4997.343549,9618.847651,1835.400914,3534.540622
S15_1,NaN,NaN,1079.508519,1282.365660,NaN,6741.611351,2770.498470,44166.663715,155.791159,NaN,...,NaN,2378.896459,578.775415,NaN,155.612310,2164.462213,931.863788,NaN,NaN,66.898734
S15_2,NaN,NaN,431.910059,2969.991594,40.520419,4879.921658,6582.420496,44739.004745,168.413145,NaN,...,NaN,437.562520,11.890336,1.767177,1485.254178,73.869226,1947.608046,NaN,NaN,282.381851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S72_2,NaN,NaN,431.013514,6979.874238,81.600816,13905.296771,4509.621834,19594.896975,33.998957,NaN,...,7049.366359,NaN,0.910518,1273.048903,3601.773523,NaN,1977.063965,NaN,3938.848802,144.146236
S72_3,NaN,NaN,NaN,11686.734568,578.499011,5685.155811,7575.763372,13276.697735,901.326825,NaN,...,5875.548618,NaN,0.355728,NaN,15046.764100,NaN,5777.303342,2325.905903,4675.475887,1257.137318
S77_1,NaN,NaN,1498.588550,1661.569130,NaN,6204.357131,2461.881926,32318.352115,NaN,NaN,...,NaN,2098.000087,3969.010070,NaN,81.449737,11647.571400,NaN,NaN,0.896501,239.696676
S77_2,NaN,NaN,693.668181,4013.370491,10.549789,4008.738822,4434.702672,27884.062105,164.837670,NaN,...,NaN,349.842049,782.966357,NaN,486.134948,357.984994,20.767380,NaN,NaN,351.841117


### Building datasets database

Atomic mass of the most common naturally occurring isotope of the more important elements in metabolites and their isotopic abundances.

In [26]:
datasets = {}
# From the alignments_new store

# GD_neg_global2 (GDg2-)
data_df = pd.HDFStore('alignments_new.h5').get('all_1ppm_min2_neg').transpose()
gd_labels = mtl.parse_data(data_df, labels_loc='label').sample_labels
new_cols_df = data_df.columns + chemdict['H'][0] - electron_mass
data_df.columns = new_cols_df

datasets['GD_neg_global2'] = {'source': 'grapevine',
                              'alignment': '1-2',
                              'mode': '-',
                              'name': 'GDg2-',
                              'data': data_df,#.to_dict(orient='split'),
                              'original': data_df,#.to_dict(orient='split'),
                              'target': gd_labels,
                              'classes': list(pd.unique(gd_labels))}

# GD_pos_global2 (GDg2+)
data_df = pd.HDFStore('alignments_new.h5').get('all_1ppm_min2_pos').transpose()
labels = mtl.parse_data(data_df, labels_loc='label').sample_labels
new_cols_df = data_df.columns - chemdict['H'][0] + electron_mass
data_df.columns = new_cols_df

datasets['GD_pos_global2'] = {'source': 'grapevine',
                              'alignment': '1-2',
                              'mode': '+',
                              'name': 'GDg2+',
                              'data': data_df,#.to_dict(orient='split'),
                              'original': data_df,#.to_dict(orient='split'),
                              'target': gd_labels,
                              'classes': list(pd.unique(gd_labels))}

# GD_neg_class2 (GDc2-)
data_df = pd.HDFStore('alignments_new.h5').get('groups_1ppm_min2_all_1ppm_neg').transpose()
labels = mtl.parse_data(data_df, labels_loc='label').sample_labels
new_cols_df = data_df.columns + chemdict['H'][0] - electron_mass
data_df.columns = new_cols_df

datasets['GD_neg_class2'] = {'source': 'grapevine',
                              'alignment': '2-1',
                              'mode': '-',
                              'name': 'GDc2-',
                              'data': data_df,#.to_dict(orient='split'),
                              'original': data_df,#.to_dict(orient='split'),
                              'target': gd_labels,
                              'classes': list(pd.unique(gd_labels))}

# GD_pos_class2 (GDc2+)
data_df = pd.HDFStore('alignments_new.h5').get('groups_1ppm_min2_all_1ppm_pos').transpose()
labels = mtl.parse_data(data_df, labels_loc='label').sample_labels
new_cols_df = data_df.columns - chemdict['H'][0] + electron_mass
data_df.columns = new_cols_df

datasets['GD_pos_class2'] = {'source': 'grapevine',
                              'alignment': '2-1',
                              'mode': '+',
                              'name': 'GDc2+',
                              'data': data_df,#.to_dict(orient='split'),
                              'original': data_df,#.to_dict(orient='split'),
                              'target': gd_labels,
                              'classes': list(pd.unique(gd_labels))}

# YD (YD)
yeast_labels = [item for item in yeast_classes for i in range(3)]

datasets['YD'] = {'source': 'yeast',
                            'alignment': '1-2',
                            'mode': '+',
                            'name': 'YD',
                            'data': yeast_datamatrix,#.to_dict(orient='split'),
                            'original': yeast_datamatrix,#.to_dict(orient='split'),
                            'target': yeast_labels,
                            'classes': list(pd.unique(yeast_labels))}

# vitis_types (GD types)
vitis_types = {'CAN': 'wild', 'RIP': 'wild', 'ROT': 'wild','RU': 'wild', 'LAB': 'wild',
               'SYL': 'wild','REG': 'vinifera','CS': 'vinifera','PN': 'vinifera','RL': 'vinifera',
               'TRI': 'vinifera'}

gd_type_labels = [vitis_types[lbl] for lbl in gd_labels]

datasets['vitis_types'] = {'source': 'grapevine',
                            'alignment': '2-1',
                            'mode': '-',
                            'name': 'GD types',
                            'data': datasets['GD_neg_class2']['original'],
                            'original': datasets['GD_neg_class2']['original'],
                            'target': gd_type_labels,
                            'classes': list(pd.unique(gd_type_labels))}

# HD (HD)
datasets['HD'] = {'source': 'human',
                  'alignment': '1-2',
                  'mode': '+',
                  'name': 'HD',
                  'data': human_datamatrix,#.to_dict(orient='split'),
                  'original': human_datamatrix,#.to_dict(orient='split'),
                  'target': hd_labels,
                  'classes': list(pd.unique(hd_labels))}


print('target for grapevine 11-variety data sets')
print(datasets['GD_neg_global2']['target'])
print('------\ntarget for 4 yeast strains data set')
print(datasets['YD']['target'])
print('------\ntarget for 2-class wild Vitis vs Vitis vinifera data set')
print(datasets['vitis_types']['target'])
print('------\ntarget for human 2-class dataset')
print(datasets['HD']['target'])

target for grapevine 11-variety data sets
['CAN', 'CAN', 'CAN', 'CS', 'CS', 'CS', 'LAB', 'LAB', 'LAB', 'PN', 'PN', 'PN', 'REG', 'REG', 'REG', 'RIP', 'RIP', 'RIP', 'RL', 'RL', 'RL', 'ROT', 'ROT', 'ROT', 'RU', 'RU', 'RU', 'SYL', 'SYL', 'SYL', 'TRI', 'TRI', 'TRI']
------
target for 4 yeast strains data set
['WT', 'WT', 'WT', 'ΔGRE3', 'ΔGRE3', 'ΔGRE3', 'ΔENO1', 'ΔENO1', 'ΔENO1', 'ΔGLO1', 'ΔGLO1', 'ΔGLO1', 'ΔGLO2', 'ΔGLO2', 'ΔGLO2']
------
target for 2-class wild Vitis vs Vitis vinifera data set
['wild', 'wild', 'wild', 'vinifera', 'vinifera', 'vinifera', 'wild', 'wild', 'wild', 'vinifera', 'vinifera', 'vinifera', 'vinifera', 'vinifera', 'vinifera', 'wild', 'wild', 'wild', 'vinifera', 'vinifera', 'vinifera', 'wild', 'wild', 'wild', 'wild', 'wild', 'wild', 'wild', 'wild', 'wild', 'vinifera', 'vinifera', 'vinifera']
------
target for human 2-class dataset
['Recurrence', 'Recurrence', 'Recurrence', 'Recurrence', 'Recurrence', 'Recurrence', 'Recurrence', 'Recurrence', 'Recurrence', 'Recurrence'

In [27]:
#fname = 'store_files/datasets.json'
#with open(fname, "w", encoding='utf8') as write_file:
    #json.dump(datasets, write_file)

In [28]:
# should not raise AssertionError:
#assert_frame_equal(yeast_datamatrix, pd.DataFrame(datasets['YD']['data']['data'], index=datasets['YD']['data']['index'], columns=datasets['YD']['data']['columns']))

In [29]:
#data_df = pd.HDFStore('alignments_new.h5').get('groups_1ppm_min2_all_1ppm_pos').transpose()
#labels = mtl.parse_data(data_df, labels_loc='label').sample_labels
#new_cols_df = data_df.columns - chemdict['H'][0] + electron_mass
#data_df.columns = new_cols_df

In [30]:
# should not raise AssertionError:
#assert_frame_equal(df, data_df)

### Writing a .csv file for each of the datasets with the list of masses that will be used to build the MDiNs (in Cytoscape's MetaNetter)

The file needs to be correctly read by Cytoscape's MetaNetter (and Cytoscape) where the network will be built.

For this, files with two columns both with the same list of masses (one index, one in the column) will be made. One will correspond to the network nodes names as a 'string'. The other will be an attribute of the corresponding node as a 'float' and will be used in the building process.

In [31]:
for dskey, ds in datasets.items():
    #if dskey.startswith('YD'):
    #    ds['data'] = pd.DataFrame(ds['data']['data'], index=ds['data']['index'], columns=ds['data']['columns'])
    #    ds['original'] = ds['data']
        
    if dskey.startswith('HD'):
        #ds['data'] = pd.DataFrame(ds['data']['data'], index=ds['data']['index'], columns=ds['data']['columns'])
        #ds['original'] = ds['data']
        
        # Specific mass lists for HD 
        # (two pairs of masses with same m/z values and different retention times are only treated as 2 different m/z)
        final_mz_list = set(final_mz_list)
        pd.DataFrame(final_mz_list, index=final_mz_list).to_csv('mass_data/MassList' + dskey + '.csv')
        continue
    
    #else:
        #df_idx = pd.MultiIndex.from_tuples(ds['data']['index'])
        #df_data = pd.DataFrame(ds['data']['data'], index=df_idx, columns=ds['data']['columns'])
        #df_data.index.set_names('label', level=0, inplace=True)
        
        #ds['data'] = df_data
        #ds['original'] = df_data
    
    # Creating mass lists in .csv files for MDiN building
    pd.DataFrame(ds['data'].columns, index=ds['data'].columns).to_csv('mass_data/MassList' + dskey + '.csv')

### Dataset Characteristics

Building a table with general characteristics about the 7 datasets studied.

In [32]:
def characterize_dataset(dskey, ds):
    "Computes and returns some general characteristics of a dataset in a dictionary."

    dataset_chrs = {}
    
    name = ds['name'] # Name of the dataset
    n_samples, n_feats = ds['data'].shape
    n_classes = len(ds['classes'])
       
    Feat_Sample = ds['data'].count(axis=1) # Nº Features in each sample
    Min_Feat_Sample = str(Feat_Sample.min()) # Minimum nº Features in a sample
    Max_Feat_Sample = str(Feat_Sample.max()) # Maximum nº Features in a sample
    Average_Feat_Sample = Feat_Sample.mean() # Average nº Features in a sample
    
    avg_feat_per_sample = int(round(Average_Feat_Sample,0)) # Round
    
    Samp_Class = len(ds['target'])/len(ds['classes']) # Nº Sample per Class
    if dskey == 'vitis_types':
        Samp_Class = '15 Vitis vinifera, 18 Wild'
        #Samp_Class = '15 $\it{Vitis}$ $\it{Vinifera}$, 18 Wild'
    elif dskey == 'HD':
        Samp_Class = '114 Recurrence, 135 no Recurrence'
    else:
        Samp_Class = str(int(Samp_Class))
    
    n_na = ds['data'].isna().sum().sum() # Nº of missing values in the dataset
    
    p_na = round(100.0 * n_na / (n_samples * n_feats), 2) # % of missing values in the dataset
    
    avg_na_per_feature = (n_samples - ds['data'].count(axis=0)).mean()
    avg_na_per_feature = int(round(avg_na_per_feature, 0))
    
    return {'Data set': name,
            '# samples': n_samples,
            '# features': n_feats,
            'features / sample (range)': f'{avg_feat_per_sample} ({Min_Feat_Sample}-{Max_Feat_Sample})',
            '# classes': n_classes,
            'samples / class':Samp_Class,
            '% missing values': p_na,} 
            #'missing values / feature': avg_na_per_feature}

data_characteristics = [characterize_dataset(dskey, ds) for dskey, ds in datasets.items()]
data_characteristics = pd.DataFrame(data_characteristics).set_index('Data set')
data_characteristics

,# samples,# features,features / sample (range),# classes,samples / class,% missing values
Data set,,,,,,
GDg2-,33,3629,658 (367-1002),11,3,81.86
GDg2+,33,7026,1164 (355-2141),11,3,83.43
GDc2-,33,3026,547 (338-919),11,3,81.91
GDc2+,33,4565,824 (215-1670),11,3,81.94
YD,15,1893,646 (559-705),5,3,65.88
GD types,33,3026,547 (338-919),2,"15 Vitis vinifera, 18 Wild",81.91
HD,249,12869,7936 (7057-8475),2,"114 Recurrence, 135 no Recurrence",38.33


In [33]:
#data_characteristics.to_excel('paperimages/dataset_characteristics.xlsx', index=True)

## Sample Mass-Difference Network Construction

#### MDBs (Mass-Difference-based Building blocks) 

Building the **list of MDBs** to use when building the MDiNs.

MDB - Mass-Difference-based Building blocks.

The choice of this MDBs is then crucial in the network building process. Ideally, they should represent different 'simple' biochemical reactions that cover the most common and ubiquitous enzymatic and non-enzymatic reactions while also being a relatively small amount of reactions – a total of 15 were picked - and maintaining the metabolite formula charge neutrality. 

For example, to maintain neutrality, a phosphorylation would mean the overall addition of a PO3H – addition of a -PO3$^{2-}$ group + 2 H$^+$ (maintaining neutrality) to replace an H atom in a metabolite. All the MDBs chosen represent changes in metabolites of no more than 5 atoms and less than 80 Da (small size). Each MDB should represent a set of chemically known reactions and a change in every main element in metabolites (C, H, O, N, S and P) is represented by at least one of the MDBs. To fulfil these conditions, representative MDBs were searched using BRENDA (https://www.brenda-enzymes.org/). The groups chosen were the following:

- CH2 (methylation) 
- O (oxygenation) 
- H2 (Hydrogenation)
- O(N-H-) (Aminase): NH3(O-) - H2
- PO3H (phosphorylation)
- NH3(O-) (transaminases)
- SO3 (sulphation)
- CO (like formylation) 
- CO2 (carboxylation, decarboxylation)
- CHOH (Hydroxymethylation) 
- NCH (formidoyltransferase)
- CONH (carbamoyltransferase)
- C2H2O (acetylation)
- S (rare but an extra S reaction)
- H2O

There could be many other MDB representing other reactions that can also be included such as CN2H2 (amidinotransferases), COCH2COO (malonyl transferases), etc.

Functions to transform formulas in string format to dictionary format and from calculate formulas (in dictionary format) exact masses.

Example:

    'String' format: 'C6H1206'
    
    'Dictionary' format: {'C':6, 'H':12, 'O':6}

In [34]:
def formula_process(formula):
    """Transforms a formula in string format into a dictionary."""

    # Empty dictionary to store the results
    results = dict.fromkeys(['C','H','O','N','S','P','Cl','F'], 0)
    count = ''
    letter = None
    minus = False
    
    # Run through the string
    for i in range(len(formula)):
        if formula[i].isupper(): # If i is an uppercase letter then it is an element
            if letter: # Just to account for the first letter in the formula where letter is None 
                # Reached another letter, store previous results and reset count
                if minus:
                    results[letter] = - int(count or 1)
                    minus = False
                else:
                    results[letter] = int(count or 1)
                count = ''
                
            if i+1 < len(formula): # In case it's a two letter element such as Cl
                if formula[i+1].islower(): # The second letter is always lower case
                    letter = formula[i] + formula[i+1] # Store new 2 letter element
                    continue
                    
            letter = formula[i] # Store new 1 letter element
            
        elif formula[i].isdigit():
            count = count + formula[i] # If number, add number to count
        
        elif formula[i] == '-':
            minus = True
    
    # Store results of the last letter
    if minus:
        results[letter] = - int(count or 1)
        minus = False
    else:
        results[letter] = int(count or 1)
                    
    return results

In [35]:
def getmass(c,h,o,n,s,p,cl,f):
    "Get the exact monoisotopic mass for any formula with the C,H,O,N,S,P,Cl and F elements only."
    massC = chemdict['C'][0] * c
    massH = chemdict['H'][0] * h
    massO = chemdict['O'][0] * o
    massN = chemdict['N'][0] * n
    massS = chemdict['S'][0] * s
    massP = chemdict['P'][0] * p
    massCl = chemdict['Cl'][0] * cl
    massF = chemdict['F'][0] * f 

    massTotal = massC + massH + massO + massN + massS + massP + massCl + massF

    return massTotal

#### Making the transformation file with MDBs to use in Cytoscape

The transformation file cwill be in a format accepted by Cytoscape's MetaNetter (.txt separated by tabs) to be considered as a list of transformations (with specific masses) to make the MDiN. 4 columns: 'Label','Formula','Mass','Selected' with mass being the most important.

Formula changes will be put under the 'Label' column and the corresponding set of reactions they represent will be put under 'Formula'. This switch is to make the MDB Impact analysis done later down the line more legible (since 'formula changes' are easier to observe, quickly understood and more concise in relation to long-winded names for set of reactions).

The 'Selected' column will be just a set of True for the MDBs to come in pre-selected in Cytoscape.

In [36]:
# Prepare DataFrame
trans_groups = pd.DataFrame(columns=['Label','Formula','Mass','Selected'])
trans_groups = trans_groups.set_index('Label')

# Chemical Formula transformations (MDBs chosen)
MDBs = ['H2','CH2','CO2','O','CHOH','NCH','O(N-H-)','S','CONH','PO3H','NH3(O-)','SO3','CO', 'C2H2O', 'H2O']
# Examples of the set of Reactions each MDB represents
labels = ['Hydrogenation','Methylation','Carboxylation/Decarboxylation','Oxygenation','Hydroxymethylation',
          'Formidoylation','Deamination','Tranfer of and -SH group','Carbamoylation','Phosphorylation','Transamination',
          'Sulphation','Formylation','Acetylation','Condensation/Dehydration/Cyclization']

for i in range(len(MDBs)): # Passing through each transformation/MDB

    # Calculate the exact mass of the MDBs
    form = formula_process(MDBs[i])
    MDB_mass = getmass(form['C'],form['H'],form['O'],form['N'],form['S'],form['P'],form['Cl'],form['F'])
    
    # MDBs with removed atoms have to be done manually (formula_process function doesn't account for this since normal
    # formulas don't have a negative amount of atoms of any elements)
    #if MDBs[i] == 'O(-NH)':
    #    MDB_mass = getmass(a['C'],-1,a['O'],-1,a['S'],a['P'],a['Cl'],a['F'])
    #elif MDBs[i] == 'NH3(-O)':
    #    MDB_mass = getmass(a['C'],a['H'],-1,a['N'],a['S'],a['P'],a['Cl'],a['F'])
        
    # Making the row for each transformation
    string = MDBs[i]
    trans_groups.loc[string] = (labels[i], MDB_mass, 'true')

trans_groups = trans_groups.sort_values(by='Mass')
trans_groups # Transformations DataFrame

,Formula,Mass,Selected
Label,,,
O(N-H-),Deamination,0.984016,true
NH3(O-),Transamination,1.031634,true
H2,Hydrogenation,2.015650,true
CH2,Methylation,14.015650,true
O,Oxygenation,15.994915,true
H2O,Condensation/Dehydration/Cyclization,18.010565,true
NCH,Formidoylation,27.010899,true
CO,Formylation,27.994915,true
CHOH,Hydroxymethylation,29.002740,true


In [37]:
#Saving the df as a .txt file in the described format
trans_groups.to_csv('transgroups.txt', header=False, sep='\t')

### Build the Mass-Difference Networks in Cytoscape using MetaNetter 2.0

To build the network in Cytoscape we'll need:

1) A list of neutral masses (with masses as float in node attributes).

2) A list of allowed transformations with specific mass differences - list of MDBs that we build above.

The transformation files and the dataset files were used in Cytoscape to build the MDiNs.

Parameters: 1 ppm error allowed for edge establishment.

The Yeast datasets already have the m/z 'buckets' that are already representing the neutral masses of the metabolites. To consider the neutral masses of the grapevine datasets, a simple transformation by adding or removing a proton (Hydrogen atom mass - electron mass), depending if the ionization mode is negative or positive, respectively, on the m/z peaks was made.


Only one full network (for each benchmark dataset) is built. Then, subgraphs of them will be used to select every sample MDiN. This is the same as building an MDiN for each sample.

The networks were exported in graphml format that networkX module can read.

- Nodes have a standard number ID instead of the mass (stored as the attribute 'mass'). Other attributes stored are irrelevant. 
- Edges among the different attributes have a very useful attribute called 'Transformation' which stores which MDB of the list was used to establish the edge - will be used for MDB Impact analysis.
- Finally, the graph is directed. Since reactions are bidireccional, they will be transformed to undirected graphs.

Changes that will be made to the network:

- Nodes will be identified by their masses.
- Intensities of the node in each sample will be given in other notebooks to store for each specific subgraph.

## Data transformations and pre-treatments

Each dataset is transformed by 6 combinations of missing value imputation and data pre-treatment methods (generating 6 data sets with different treatments applied).

### Traditional and more established  intensity-based Pre-Treatments

Missing value imputation is mandatory for the data sets since many statistical methods performed downstream can't handle missing values. 

Two missing value imputation procedures were applied before any intensity-based pre-treatments: 

- Missing values of a sample were replaced with a fifth of the minimum intensity value present in that sample of the data matrix - Limit of Detection type of imputation. This was performed with the `fillna_frac_sample_min`function below. 

- Missing values were imputed by Random Forest missing value imputation. Parameters: 10 trees, 100 nearest features considered, 0 minimum value, 1.0e10 maximum value and 2 max iterations, others were left as default. This was performed by using the scikit-learn Python module with `IterativeImputer`and `ExtraTreesRegressor` as estimator. In cases where this method was used, an extra **`_RF`** was added to the 'name' of the pre-treatment.

#### Combinations of intensity-based pre-treatments made:

- Ionly Treatment - Only Missing Value Imputation.

- P Treatment - Missing Value Imputation and Pareto Scaling.

- NP Treatment - Missing Value Imputation, Normalization and Pareto Scaling.

- **NGP Treatment** - Missing Value Imputation, Normalization, Generalized Logarithmic Transformation and Pareto Scaling.

For normalizations, Normalization by reference feature (Leucine Enkephalin) is used in all cases, except for the HD dataset where Probabilistic Quotient Normalization (PQN) is used with the mean of all samples acting as the reference sample.

Note: Leucine Enkephalin peak (reference feature) is removed upon normalization. Order of pre-treatments is the order in which they were mentioned.

In [38]:
def impute_RF(df, nearest_features=100, n_trees=50):
    "Random Forest Imputation of Missing Values."
    rf_estimator = ExtraTreesRegressor(n_estimators=n_trees)
    imputer = IterativeImputer(random_state=0, estimator=rf_estimator,
                           n_nearest_features=nearest_features,
                           min_value=0.0, max_value=1.0e10,
                           max_iter=2,
                           verbose=0)
    imputed_data = imputer.fit_transform(df)
    res = pd.DataFrame(imputed_data, index=df.index, columns=df.columns)
    ncols = len(res.columns)
    res = res.dropna(axis='columns', how='any')
    ncols2 = len(res.columns)
    if ncols > ncols2:
        print(f'{ncols-ncols2} features dropped')
    return res

In [39]:
# For missing value imputation based on constants relative to the minimum of each feature instead of the full dataset
def fillna_frac_feat_min(df, fraction=0.2):
    """Set NaN to a fraction of the minimum value in each column of the DataFrame."""

    minimum = df.min(axis=0) * fraction
    return df.fillna(minimum)

# For missing value imputation based on constants relative to the minimum of each sample instead of the full dataset
def fillna_frac_sample_min(df, fraction=0.2):
    """Set NaN to a fraction of the minimum value in each column of the DataFrame."""

    minimum = df.min(axis=1) * fraction
    return df.T.fillna(minimum).T

In [40]:
# Performs all pre-treatment combinations mentioned
def compute_transf(dataset, norm_ref=None, lamb=None):
    "Computes 6 combinations of pre-treatments and missing value imputations and returns after treatment datasets in a dict."
    
    data = dataset['data']
    
    # Imputation of Missing Values
    #imputed = fillna_frac_feat_min(data, fraction=0.2)
    imputed = fillna_frac_sample_min(data, fraction=0.2)
    
    # Imputation by RF
    imputedRF = impute_RF(data, nearest_features=100, n_trees=10)
    
    
    # Normalization by a reference feature
    if norm_ref is not None:
        norm = transf.normalize_ref_feature(imputed, norm_ref, remove=True)
    else:
        norm = imputed
    
    # Normalization by a reference feature RF
    if norm_ref is not None:
        normRF = transf.normalize_ref_feature(imputedRF, norm_ref, remove=True)
    else:
        normRF = imputedRF
    
    # Normalization by PQN for HD - 1/2 and RF   
    if dataset['name'].startswith('HD'):
        norm = transf.normalize_PQN(imputed, ref_sample='mean')
        normRF = transf.normalize_PQN(imputedRF, ref_sample='mean')

    
    # Pareto Scaling and Generalized Logarithmic Transformation
    P = transf.pareto_scale(imputed)
    NP = transf.pareto_scale(norm)
    NGP = transf.pareto_scale(transf.glog(norm, lamb=lamb))

    # Pareto Scaling and Generalized Logarithmic Transformation
    P_RF = transf.pareto_scale(imputedRF)
    NP_RF = transf.pareto_scale(normRF)
    NGP_RF = transf.pareto_scale(transf.glog(normRF, lamb=lamb))
    
    # Store results
    dataset['Ionly'] = imputed
    dataset['P'] = P
    dataset['NP'] = NP
    dataset['NGP'] = NGP
    
    dataset['Ionly_RF'] = imputedRF
    dataset['P_RF'] = P_RF
    dataset['NP_RF'] = NP_RF
    dataset['NGP_RF'] = NGP_RF    

Human leucine enkephalin (Sigma Aldrich) is the reference feature (internal standard, [M+H]+ = 556.276575 Da or [M-H]- = 554.262022 Da) used for these datasets.

Search in the grapevine data sets for the reference feature and confirm the reference feature in the yeast data sets with `find_closest_features` from metabolinks package.

There is **no** reference feature in the HD dataset.

In [41]:
# Theoretical mass for negative mode Leucine Enkephalin - 554.262022
# Theoretical mass for positive mode Leucine Enkephalin - 556.276575
Leu_Enk_neg = 554.262022 + chemdict['H'][0] - electron_mass
Leu_Enk_pos = 556.276575 - chemdict['H'][0] + electron_mass

# Reference Feature in the yeast dataset
leu_enk_name = 555.2692975341

In [42]:
query_datasets = [name for name in datasets if datasets[name]['source']=='grapevine']

for name in query_datasets:
    ds = datasets[name]
    print(f'looking for reference in {name} ...')
    ref_variable = Leu_Enk_neg if ds['mode'] == '-' else Leu_Enk_pos
    closest = transf.find_closest_features(ds['data'], features=[ref_variable])
    if closest[ref_variable] is not None:
        print('Found ref feature', ref_variable)
        delta = closest[ref_variable] - ref_variable
        print(f'In data: {closest[ref_variable]} delta = {delta:.3e}\n')

query_datasets = [name for name in datasets if name.startswith('YD')]

ref_variable = leu_enk_name
for name in query_datasets:
    ds = datasets[name]
    print(f'looking for reference in {name} ...') 
    closest = transf.find_closest_features(ds['data'], features=[ref_variable])
    if closest[ref_variable] is not None:
        print('Found ref feature', ref_variable, '\n')
    else:
        print('Ref feature not found\n')

looking for reference in GD_neg_global2 ...
Found ref feature 555.269298452291
In data: 555.269296452291 delta = -2.000e-06

looking for reference in GD_pos_global2 ...
Found ref feature 555.269298547709
In data: 555.269293547709 delta = -5.000e-06

looking for reference in GD_neg_class2 ...
Found ref feature 555.269298452291
In data: 555.269296452291 delta = -2.000e-06

looking for reference in GD_pos_class2 ...
Found ref feature 555.269298547709
In data: 555.269293547709 delta = -5.000e-06

looking for reference in vitis_types ...
Found ref feature 555.269298452291
In data: 555.269296452291 delta = -2.000e-06

looking for reference in YD ...
Found ref feature 555.2692975341 



##### Apply the different pre-treatments and get the results in their respective dictionaries

In [43]:
for name, ds in datasets.items():
    dataset_name = datasets[name]["name"]
    print(f'Applying pre-processing transformations to data in {dataset_name}', end=' ...')
    start = perf_counter()
    
    if name.startswith('YD'):
        ref_variable = leu_enk_name
    elif name.startswith('HD'):
        ref_variable = None
    else:
        ref_variable = Leu_Enk_neg if ds['mode'] == '-' else Leu_Enk_pos
    
    compute_transf(ds, norm_ref=ref_variable)
    
    end = perf_counter()

    print(f'done! took {(end - start):.3f} s')

Applying pre-processing transformations to data in GDg2- ...

C:\Users\francisco\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


done! took 76.879 s
Applying pre-processing transformations to data in GDg2+ ...

C:\Users\francisco\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


done! took 145.294 s
Applying pre-processing transformations to data in GDc2- ...

...

done! took 33.743 s
Applying pre-processing transformations to data in GD types ...

C:\Users\francisco\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


done! took 57.882 s
Applying pre-processing transformations to data in HD ...

C:\Users\francisco\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


done! took 869.336 s


### Generate json files

In [44]:
# ensure dir exists
path = Path.cwd() / "store_files"
path.mkdir(parents=True, exist_ok=True)

storepath = Path.cwd() / "store_files" / 'processed_data.h5'

store = pd.HDFStore(storepath, complevel=9, complib="blosc:blosclz")
#pd.set_option('io.hdf.default_format','table')

# keep json serializable values and store dataFrames in HDF store

serializable = {}

for dskey, dataset in datasets.items():
    serializable[dskey] = {}
    for key, value in dataset.items():
        #print(dskey, key)
        if isinstance(value, pd.DataFrame):
            storekey = dskey + '_' + key
            #print('-----', storekey)
            store[storekey] = value
            serializable[dskey][key] = f"INSTORE_{storekey}"
        else:
            serializable[dskey][key] = value
store.close()
            

path = path / 'processed_data.json'
with open(path, "w", encoding='utf8') as write_file:
    json.dump(serializable, write_file)

#serializable